In [1]:
from kamiwaza_client import KamiwazaClient
import openai
from pydantic import BaseModel

In [2]:
client = KamiwazaClient("http://localhost:7777/api/")
client.serving.list_active_deployments()

[ActiveModelDeployment(id=UUID('acae8693-1373-4595-bdaa-7570d2fe9c05'), m_id=UUID('fce9ec25-c030-4db9-94f7-a4ee74cde344'), m_name='Qwen3-8B-GGUF', status='DEPLOYED', instances=[ModelInstance:
 ID: 73918ccc-a1e1-4e2b-8858-a28fb85c70e0
 Deployment ID: acae8693-1373-4595-bdaa-7570d2fe9c05
 Status: DEPLOYED
 Listen Port: 50375], lb_port=61100, endpoint='http://localhost:61100/v1')]

In [3]:
openai_client = client.openai.get_client('Qwen3-8B-GGUF')

## Define the runtime tool & its JSON schema
We expose a dummy weather function so the demo is entirely self‑contained.


In [4]:
from typing import Dict, Any
import json, time
from pydantic import BaseModel, Field

class WeatherParams(BaseModel):
    city: str = Field(..., description="City name")
    state: str = Field(..., description="US state abbreviation, e.g. CA")
    unit: str = Field(..., description="Either 'celsius' or 'fahrenheit'")

def get_current_weather(city: str, state: str, unit: str) -> Dict[str, Any]:
    """Pretend weather service so we don't need a real API call."""
    fake_temp_f = 100  # you could randomise this or call a real API
    temp = fake_temp_f if unit == "fahrenheit" else round((fake_temp_f - 32) * 5 / 9)
    time.sleep(0.3)  # small pause so the streaming looks lively
    return {
        "temperature": temp,
        "unit": unit,
        "city": city,
        "state": state,
        "description": "clear skies with a light breeze",
    }

weather_tool_schema = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given US city",
        "parameters": WeatherParams.schema(),  # Pydantic → JSON Schema
    },
}

tools = [weather_tool_schema]

local_tool_registry = {"get_current_weather": get_current_weather}
print("✅ Tool registered — ready for the chat call")


✅ Tool registered — ready for the chat call


/var/folders/yj/9j0gvmw94m78t21q_rkctb340000gn/T/ipykernel_38680/2537202073.py:28: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  "parameters": WeatherParams.schema(),  # Pydantic → JSON Schema


## Kick off a streaming chat with tool support
The system prompt asks the model to *think in public* using `<thinking>` 
tags, then decide whether to call the weather tool.


In [5]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a helpful assistant. When you receive tool outputs, "
            "always provide a complete and helpful final answer to the user's question. "
            "Be conversational and provide practical advice."
        ),
    },
    {
        "role": "user",
        "content": "should i wear short sleeves or long sleeves in nyc today?",
    },
]

print("\n📡 **Streaming** — watch the tokens, tool call, and final answer arrive\n")

# First API call with tools
response_stream = openai_client.chat.completions.create(
    model="model", 
    messages=messages,
    tools=tools,
    tool_choice="auto",  # Let the model decide when to use tools
    stream=True,
)

# Track tool calls as they stream in
tool_calls = []
current_tool_call = None
accumulated_content = ""

for chunk in response_stream:
    if not chunk.choices:
        continue
    
    choice = chunk.choices[0]
    delta = choice.delta
    
    # Handle reasoning content (if your model supports it)
    if hasattr(delta, "reasoning_content") and delta.reasoning_content:
        print("\033[36m" + delta.reasoning_content + "\033[0m", end="", flush=True)
    
    # Handle regular content
    if delta.content:
        accumulated_content += delta.content
        print(delta.content, end="", flush=True)
    
    # Handle tool calls
    if delta.tool_calls:
        for tc_delta in delta.tool_calls:
            # Start a new tool call
            if tc_delta.index == 0 and not current_tool_call:
                current_tool_call = {
                    "id": tc_delta.id or f"call_{uuid.uuid4().hex[:8]}",
                    "type": "function",
                    "function": {
                        "name": tc_delta.function.name,
                        "arguments": ""
                    }
                }
            
            # Accumulate arguments
            if tc_delta.function.arguments:
                current_tool_call["function"]["arguments"] += tc_delta.function.arguments

# Execute tool calls if any
if choice.finish_reason == "tool_calls" and current_tool_call:
    print(f"\n\n🔧 Model requested tool: {current_tool_call['function']['name']}")
    
    # Parse arguments and execute
    args = json.loads(current_tool_call["function"]["arguments"])
    print(f"   with args: {args}")
    
    tool_name = current_tool_call["function"]["name"]
    tool_result = local_tool_registry[tool_name](**args)
    print(f"🔧 Tool response: {tool_result}")
    
    # Add the assistant's tool call message
    messages.append({
        "role": "assistant",
        "content": accumulated_content if accumulated_content else None,
        "tool_calls": [current_tool_call]
    })
    
    # Add the tool response
    messages.append({
        "role": "tool",
        "tool_call_id": current_tool_call["id"],
        "name": tool_name,
        "content": json.dumps(tool_result)
    })
    
    # Make a follow-up call to get the final answer
    print("\n\n💭 Getting final answer...\n")
    
    follow_stream = openai_client.chat.completions.create(
        model="model",
        messages=messages,
        stream=True,
        # No tools this time - we want a final answer
    )
    
    # Stream the final response
    for follow_chunk in follow_stream:
        if not follow_chunk.choices:
            continue
        
        follow_choice = follow_chunk.choices[0]
        follow_delta = follow_choice.delta
        
        # Handle reasoning content
        if hasattr(follow_delta, "reasoning_content") and follow_delta.reasoning_content:
            print("\033[36m" + follow_delta.reasoning_content + "\033[0m", end="", flush=True)
        
        # Handle final answer content
        if follow_delta.content:
            print(follow_delta.content, end="", flush=True)
    
    print("\n\n🎉 **Done!**")

# If no tool was called, we're done
elif choice.finish_reason == "stop":
    print("\n\n🎉 **Done!** (No tool calls needed)")

2025-08-08 11:06:30,551 - httpx - INFO - HTTP Request: POST http://localhost:61100/v1/chat/completions "HTTP/1.1 200 OK"



📡 **Streaming** — watch the tokens, tool call, and final answer arrive

Okay, the user is asking whether they should wear short sleeves or long sleeves in NYC today. To figure this out, I need to know the current weather in NYC. The tools provided include a function called get_current_weather, which requires the city, state, and unit (celsius or fahrenheit). 

First, I should check the parameters. The user mentioned NYC, so the city is "New York" and the state is "NY". The unit isn't specified, but since the user is in the US, Fahrenheit is probably more common. However, the function allows for either, so I should specify one. Let's go with Fahrenheit as it's standard in the US.

So, I need to call get_current_weather with city: "New York", state: "NY", unit: "fahrenheit". Once I get the temperature, I can advise the user. If it's above 75°F, short sleeves would be fine. If it's cooler, maybe around 65-75, a light sweater might be good. Below 65, long sleeves would be better. But I sh

2025-08-08 11:06:38,202 - httpx - INFO - HTTP Request: POST http://localhost:61100/v1/chat/completions "HTTP/1.1 200 OK"


🔧 Tool response: {'temperature': 100, 'unit': 'fahrenheit', 'city': 'New York', 'state': 'NY', 'description': 'clear skies with a light breeze'}


💭 Getting final answer...

Okay, the user is asking whether to wear short sleeves or long sleeves in NYC today. I need to check the current weather for New York. The tool response came back with a temperature of 100°F, clear skies, and a light breeze. 

First, 100°F is quite hot. Typically, when it's that warm, short sleeves are more comfortable. But I should consider other factors. The description mentions clear skies, so probably sunny. No mention of humidity, but even if it's dry, 100°F is still very hot. A light breeze might help with the heat a bit, but it's not enough to make long sleeves necessary.

I should also think about what's typical for NYC in such weather. People usually wear light clothing. Maybe suggest short sleeves and mention staying hydrated. Also, maybe a light jacket if there's a chance of cooling down later, but the c